In [1]:
import pandas as pd
import numpy as np

In [2]:
import os
directory = 'TrainFull'
list_raw_files = []

for filename in os.listdir(directory):
    if filename.endswith(".csv"):
        path = (directory + '/' + filename)
        list_raw_files.append(path)
list_raw_files

['TrainFull/mealData1.csv',
 'TrainFull/mealData2.csv',
 'TrainFull/mealData3.csv',
 'TrainFull/mealData4.csv',
 'TrainFull/Nomeal1.csv',
 'TrainFull/Nomeal2.csv',
 'TrainFull/Nomeal3.csv',
 'TrainFull/Nomeal4.csv']

In [3]:
list_of_df = []
import csv
for i in list_raw_files:
    df = pd.read_csv(i)
    df.drop(df.columns[[0]], axis=1, inplace=True)
    list_of_df.append(df)
list_of_df

[        0      1      2      3      4      5      6      7      8      9  ...  \
 0   221.0  232.0  240.0  244.0  245.0  246.0  249.0  254.0  261.0  272.0  ...   
 1    55.0   62.0   70.0   79.0   89.0  100.0  112.0  121.0  131.0  141.0  ...   
 2   214.0  213.0  211.0  210.0  205.0  200.0  195.0  195.0  204.0  205.0  ...   
 3   226.0  224.0  228.0  222.0  214.0  206.0  199.0  194.0  190.0  184.0  ...   
 4   188.0  181.0  176.0  174.0  172.0  170.0  170.0  170.0  168.0  166.0  ...   
 5   119.0  127.0  136.0  143.0  151.0  160.0  173.0  183.0  189.0  192.0  ...   
 6   256.0  258.0  260.0  258.0  254.0  252.0  246.0  241.0  236.0  228.0  ...   
 7   307.0  301.0  289.0  285.0  274.0  266.0  259.0  255.0  242.0  230.0  ...   
 8   137.0  136.0  134.0  131.0  129.0  127.0  123.0  121.0  120.0  121.0  ...   
 9   193.0  190.0  193.0  200.0  203.0  203.0  205.0  197.0  196.0  202.0  ...   
 10   86.0   95.0  105.0  110.0  121.0  133.0  145.0  156.0  165.0  173.0  ...   
 11  329.0  328.

In [4]:
def calculate_iqr(row):
    median = row.quantile(0.5)
    quantile1 = row.quantile(0.25)
    quantile3 = row.quantile(0.75)
    iqr = quantile3 - quantile1
    return iqr

In [5]:
def calculate_ZC(row):
    zero_x = 0

    for idx, item in enumerate(row[:-1]):
        if row[idx] == row[idx+1]:
            zero_x +=1
        
    return zero_x

In [6]:
def calculate_GFI(row):
    sum_squares = 0
    for idx, item in enumerate(row[:-1]):
        currdiff = row[idx] - row[idx+1]
        sum_squares += currdiff ** 2
    return np.sqrt(sum_squares/(len(row)-1))

In [7]:
for currdf in list_of_df:
    currdf['mean'] = currdf.mean(axis=1)
    currdf['lbgv'] = currdf.min(axis=1)
    currdf['hbgv'] = currdf.max(axis=1)
    currdf['median'] = currdf.median(axis=1)
    currdf['iqr'] = currdf.apply(calculate_iqr, axis=1)
    currdf['std'] = currdf.std(axis=1)
    currdf['MAG'] = currdf.diff(axis=1).abs().sum(axis=1)/2.5
    currdf['ZC'] = currdf.apply(calculate_ZC, axis=1)
    currdf['CV'] = currdf.std(axis=1)/currdf.mean(axis=1)
    currdf['GFI'] = currdf.apply(calculate_GFI, axis=1)
    currdf['GCF'] = currdf.apply(calculate_GFI, axis=1)/currdf.mean(axis=1)
    currdf['MAGE'] = currdf.max(axis=1) - currdf.min(axis=1)

In [8]:
for idx,currdf in enumerate(list_of_df):
    if idx <= 3:
        currdf['Label'] = 1
    else:
        currdf['Label'] = 0

In [9]:
#pca for meal
feat_df_list = []

for idx,currdf in enumerate(list_of_df):
    if idx <= 3:
        newdf = currdf[['mean','lbgv','hbgv','median','iqr','std','MAG','CV','ZC','GFI','GCF','MAGE']]
        feat_df_list.append(newdf)
feat_df_list

[          mean   lbgv        hbgv      median         iqr        std  \
 0   268.000000  221.0  312.000000  268.000000   56.000000  46.590294   
 1   143.000000   55.0  202.000000  143.000000   52.000000  45.475453   
 2   144.433333   78.0  214.000000  144.433333  119.500000  55.741623   
 3   164.200000  118.0  228.000000  153.000000   57.000000  40.078106   
 4   143.500000   95.0  188.000000  156.000000   61.250000  33.752521   
 5   165.433333  119.0  192.000000  171.000000   34.500000  31.265977   
 6   170.166667   90.0  260.000000  170.166667  140.750000  66.860057   
 7   198.433333  129.0  307.000000  181.000000   99.500000  60.148437   
 8   118.800000   92.0  143.000000  124.000000   36.000000  22.348699   
 9   159.166667  115.0  205.000000  161.000000   75.250000  38.938925   
 10  232.233333   86.0  336.000000  236.000000  169.750000  86.906460   
 11  220.366667  117.0  329.000000  225.000000  125.250000  71.586230   
 12  165.333333  106.0  199.000000  170.000000   16

In [10]:
df_pca = pd.concat(feat_df_list, ignore_index=True)
df_pca

,mean,lbgv,hbgv,median,iqr,std,MAG,CV,ZC,GFI,GCF,MAGE
0,268.000000,221.0,312.000000,268.000000,56.000000,46.590294,248.963883,0.281728,1,65.653640,0.275609,311.724391
1,143.000000,55.0,202.000000,143.000000,52.000000,45.475453,243.009819,0.397342,2,60.778653,0.464177,242.612477
2,144.433333,78.0,214.000000,144.433333,119.500000,55.741623,233.250018,0.440050,3,56.759655,0.421721,232.828297
3,164.200000,118.0,228.000000,153.000000,57.000000,40.078106,203.328758,0.327926,0,50.945507,0.342960,228.000000
4,143.500000,95.0,188.000000,156.000000,61.250000,33.752521,178.098992,0.320006,2,44.876367,0.344623,187.679994
5,165.433333,119.0,192.000000,171.000000,34.500000,31.265977,203.093609,0.292795,1,51.168764,0.344999,202.800814
6,170.166667,90.0,260.000000,170.166667,140.750000,66.860057,279.789311,0.446870,2,68.888544,0.433594,279.355717
7,198.433333,129.0,307.000000,181.000000,99.500000,60.148437,306.287292,0.377241,0,76.391849,0.417895,307.000000
8,118.800000,92.0,143.000000,124.000000,36.000000,22.348699,128.900520,0.287309,3,32.547386,0.305190,142.712691
9,159.166667,115.0,205.000000,161.000000,75.250000,38.938925,188.157763,0.325646,2,45.953272,0.318742,204.681258


In [11]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
x = df_pca.values
x = StandardScaler().fit_transform(x)
col_mean = np.nanmean(x, axis=0)
inds = np.where(np.isnan(x))
x[inds] = np.take(col_mean, inds[1])

pca = PCA(n_components=5)
principalComponents = pca.fit_transform(x)
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5','principal component 6','principal component 7','principal component 8','principal component 9'])
principalDf1 = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5'])
principalDf1

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5
0,1.583310,-3.482300,-0.392497,-0.196212,-0.466898
1,-0.448770,2.134294,0.126957,0.631236,0.350344
2,0.164418,2.253369,1.272030,-0.837988,-0.310237
3,-0.892522,0.089954,-1.089431,-0.574656,-0.143123
4,-1.996459,0.475438,0.122248,-0.491524,0.322998
5,-1.673737,-0.334666,-0.709012,0.068119,0.378190
6,1.937909,1.954068,0.851981,-1.054740,-0.312819
7,2.360940,0.396524,-0.734362,-0.396082,-0.373288
8,-3.954162,0.378542,0.516285,-0.339956,0.345298
9,-1.422282,0.061475,0.269006,-0.836827,0.029939


In [12]:
#pca for non meal

feat_df_list = []
for idx,currdf in enumerate(list_of_df):
    if idx > 3:
        newdf = currdf[['mean','lbgv','hbgv','median','iqr','std','MAG','CV','ZC','GFI','GCF','MAGE']]
        feat_df_list.append(newdf)

df_pca = pd.concat(feat_df_list, ignore_index=True)
x = df_pca.values
x = StandardScaler().fit_transform(x)
col_mean = np.nanmean(x, axis=0)
inds = np.where(np.isnan(x))
x[inds] = np.take(col_mean, inds[1])
principalComponents = pca.transform(x)
#principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5','principal component 6','principal component 7','principal component 8','principal component 9'])
principalDf2 = pd.DataFrame(data = principalComponents, columns = ['principal component 1', 'principal component 2', 'principal component 3', 'principal component 4','principal component 5'])


principalDf=pd.concat([principalDf1,principalDf2],ignore_index=True)
principalDf

,principal component 1,principal component 2,principal component 3,principal component 4,principal component 5
0,1.583310,-3.482300,-0.392497,-0.196212,-0.466898
1,-0.448770,2.134294,0.126957,0.631236,0.350344
2,0.164418,2.253369,1.272030,-0.837988,-0.310237
3,-0.892522,0.089954,-1.089431,-0.574656,-0.143123
4,-1.996459,0.475438,0.122248,-0.491524,0.322998
5,-1.673737,-0.334666,-0.709012,0.068119,0.378190
6,1.937909,1.954068,0.851981,-1.054740,-0.312819
7,2.360940,0.396524,-0.734362,-0.396082,-0.373288
8,-3.954162,0.378542,0.516285,-0.339956,0.345298
9,-1.422282,0.061475,0.269006,-0.836827,0.029939


In [13]:
df_init = pd.concat(list_of_df, ignore_index=True)
df_init

,0,1,2,3,4,5,6,7,8,9,...,median,iqr,std,MAG,ZC,CV,GFI,GCF,MAGE,Label
0,221.0,232.0,240.0,244.0,245.0,246.0,249.0,254.0,261.0,272.0,...,268.000000,56.000000,46.590294,248.963883,1,0.281728,65.653640,0.275609,311.724391,1
1,55.0,62.0,70.0,79.0,89.0,100.0,112.0,121.0,131.0,141.0,...,143.000000,52.000000,45.475453,243.009819,2,0.397342,60.778653,0.464177,242.612477,1
2,214.0,213.0,211.0,210.0,205.0,200.0,195.0,195.0,204.0,205.0,...,144.433333,119.500000,55.741623,233.250018,3,0.440050,56.759655,0.421721,232.828297,1
3,226.0,224.0,228.0,222.0,214.0,206.0,199.0,194.0,190.0,184.0,...,153.000000,57.000000,40.078106,203.328758,0,0.327926,50.945507,0.342960,228.000000,1
4,188.0,181.0,176.0,174.0,172.0,170.0,170.0,170.0,168.0,166.0,...,156.000000,61.250000,33.752521,178.098992,2,0.320006,44.876367,0.344623,187.679994,1
5,119.0,127.0,136.0,143.0,151.0,160.0,173.0,183.0,189.0,192.0,...,171.000000,34.500000,31.265977,203.093609,1,0.292795,51.168764,0.344999,202.800814,1
6,256.0,258.0,260.0,258.0,254.0,252.0,246.0,241.0,236.0,228.0,...,170.166667,140.750000,66.860057,279.789311,2,0.446870,68.888544,0.433594,279.355717,1
7,307.0,301.0,289.0,285.0,274.0,266.0,259.0,255.0,242.0,230.0,...,181.000000,99.500000,60.148437,306.287292,0,0.377241,76.391849,0.417895,307.000000,1
8,137.0,136.0,134.0,131.0,129.0,127.0,123.0,121.0,120.0,121.0,...,124.000000,36.000000,22.348699,128.900520,3,0.287309,32.547386,0.305190,142.712691,1
9,193.0,190.0,193.0,200.0,203.0,203.0,205.0,197.0,196.0,202.0,...,161.000000,75.250000,38.938925,188.157763,2,0.325646,45.953272,0.318742,204.681258,1


In [14]:
from sklearn.svm import SVC

#classifier = SVC(kernel='linear')
#classifier = SVC(kernel='poly', degree=8)
#classifier = SVC(kernel='poly', degree=4)
#classifier = SVC(kernel='rbf')
classifier = SVC(kernel='sigmoid')

classifier.fit(principalDf, df_init['Label'])

C:\Users\Meghana\Anaconda3\lib\site-packages\sklearn\svm\base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='auto_deprecated',
    kernel='sigmoid', max_iter=-1, probability=False, random_state=None,
    shrinking=True, tol=0.001, verbose=False)

In [15]:
import pickle
filename = 'finalized_model.sav'
pca_filename = 'finalized_pca.sav'
pickle.dump(classifier, open(filename, 'wb'))
pickle.dump(pca, open(pca_filename, 'wb'))

In [16]:
print("hello")

hello
